In [38]:
import pandas as pd, numpy as np

In [2]:
data = pd.read_csv('../data/FUF Salesforce Data Dump.csv')

/usr/local/lib/python2.7/site-packages/pandas/io/parsers.py:1170: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  data = self._reader.read(nrows)


In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 61745 entries, 0 to 61744
Data columns (total 20 columns):
Tree Planting: Planting ID    61745 non-null object
Tree Species                  51502 non-null object
Tree Bldg Num                 59790 non-null object
Tree Street                   61371 non-null object
Tree Zip Code                 57992 non-null object
Neighborhood                  61694 non-null object
Cross Street                  41304 non-null object
Basin Type                    25983 non-null object
Permit #                      1763 non-null object
Survey2mDoneDate              4632 non-null object
Survey2mStatus                14435 non-null object
Care1Done                     27556 non-null object
2yrDone                       760 non-null object
Survey2yrStatus               747 non-null object
3yrDone                       21395 non-null object
Survey3yrStatus               18690 non-null object
4yrDone                       61 non-null object
Survey4yrStatus 

In [4]:
data.filter(regex='.*Status').count()

Survey2mStatus     14435
Survey2yrStatus      747
Survey3yrStatus    18690
Survey4yrStatus       60
Survey5yrStatus     3046
dtype: int64

In [5]:
data.filter(regex='.*Done').count()

Survey2mDoneDate     4632
Care1Done           27556
2yrDone               760
3yrDone             21395
4yrDone                61
5yrDone              3244
dtype: int64

- Status primarily available after 2m and 3yr, looks like it would be most useful to consider these two points in time

In [6]:
df_notna = data[['Tree Species', 'Neighborhood', 'Survey2mStatus', 'Survey3yrStatus']].dropna()

In [7]:
df_notna.describe()

,Tree Species,Neighborhood,Survey2mStatus,Survey3yrStatus
count,9964,9964,9964,9964
unique,243,45,9,6
top,Arbutus 'Marina',Outer Sunset,1 Very healthy,2 Good
freq,1017,855,6529,4103


In [8]:
df_notna['Survey2mStatus'].value_counts()

1 Very healthy          6529
2 Good                  2872
3 Struggling             497
4 Almost dead             56
5 Dead or gone             6
2                          1
1                          1
survey but no remark       1
-1                         1
dtype: int64

In [9]:
df_notna['Survey3yrStatus'].value_counts()

2 Good            4103
1 Very healthy    3972
3 Struggling      1224
5 Dead or gone     408
4 Almost dead      245
5 dead/gone         12
dtype: int64

In [10]:
df_notna = df_notna[~df_notna['Survey2mStatus'].isin(['-1','survey but no remark'])]

In [11]:
di = {"1": 1, "1 Very healthy": 1, "2": 2, "2 Good": 2,
     "3": 3, '3 Struggling': 3, "4": 4, '4 Almost dead': 4,
     "5": 5, '5 Dead or gone': 5, '5 dead/gone': 5}

df_notna.replace(di, inplace = True)

df_notna['2m_3yr_change'] = df_notna['Survey2mStatus'] - df_notna['Survey3yrStatus']

In [12]:
df_notna['Survey2mStatus'].value_counts()

1    6530
2    2873
3     497
4      56
5       6
dtype: int64

In [13]:
df_notna['Survey3yrStatus'].value_counts()

2    4103
1    3970
3    1224
5     420
4     245
dtype: int64

In [14]:
df_notna['2m_3yr_change'].value_counts()

 0    4148
-1    3053
 1    1270
-2     840
-3     267
-4     237
 2     136
 3      11
dtype: int64

In [15]:
df_notna['2m_3yr_change'].describe()

count    9962.000000
mean       -0.492572
std         1.102331
min        -4.000000
25%        -1.000000
50%         0.000000
75%         0.000000
max         3.000000
Name: 2m_3yr_change, dtype: float64

In [16]:
df_notna[df_notna['2m_3yr_change']==-4]['Neighborhood'].value_counts().head()

Bayview           25
Mission           24
Outer Sunset      23
Inner Sunset      16
Bernal Heights    13
dtype: int64

In [17]:
df_notna[df_notna['2m_3yr_change']==-4]['Tree Species'].value_counts().head()

Tristania laurina        17
Arbutus 'Marina'         16
Metrosideros excelsus    14
Lophostemon confertus    13
Pittosporum undulatum    13
dtype: int64

In [18]:
df_notna['2m_3yr_change_category'] = df_notna['2m_3yr_change'].apply(lambda x: 'Better or Same' if x in [0,1,2,3] else
                                                                                'Worse')
df_notna['2m_3yr_change_binary'] = df_notna['2m_3yr_change'].apply(lambda x: '1' if x in [0,1,2,3] else
                                                                                '0')

In [19]:
df_notna['2m_3yr_change_category'].value_counts()

Better or Same    5565
Worse             4397
dtype: int64

In [20]:
df_notna.groupby('2m_3yr_change_category').describe()

2m_3yr_change  Survey2mStatus  Survey3yrStatus
2m_3yr_change_category                                                      
Better or Same         count    5565.000000     5565.000000      5565.000000
                       mean        0.283019        1.594070         1.311051
                       std         0.513521        0.677512         0.516891
                       min         0.000000        1.000000         1.000000
                       25%         0.000000        1.000000         1.000000
                       50%         0.000000        1.000000         1.000000
                       75%         1.000000        2.000000         2.000000
                       max         3.000000        5.000000         5.000000
Worse                  count    4397.000000     4397.000000      4397.000000
                       mean       -1.474187        1.171253         2.645440
                       std         0.833271        0.430324         0.953209
                       min        -4.000000        1.000000         2.000000
                       25%        -2.000000        1.000000         2.000000
                       50%        -1.000000        1.000000         2.000000
                       75%        -1.000000        1.000000         3.000000
                       max        -1.000000        4.000000         5.000000

In [21]:
df_notna.ix[df_notna['Tree Species']<>'', 
            'genus'] = df_notna[df_notna['Tree Species']<>'']['Tree Species'].apply(lambda x: x.split()[0])

In [22]:
df_notna[df_notna['2m_3yr_change']==-4]['genus'].value_counts().head()

Tristania      24
Magnolia       21
Prunus         20
Arbutus        17
Pittosporum    15
dtype: int64

In [23]:
df_notna.head()

,Tree Species,Neighborhood,Survey2mStatus,Survey3yrStatus,2m_3yr_change,2m_3yr_change_category,2m_3yr_change_binary,genus
25,Prunus salicina 'Santa Rosa',Bayview,1,2,-1,Worse,0,Prunus
49,Prunus yedoensis 'Akebono',Mission,1,1,0,Better or Same,1,Prunus
52,Prunus yedoensis 'Akebono',Mission,1,1,0,Better or Same,1,Prunus
58,Rhamnus alaternus,Outer Sunset,1,1,0,Better or Same,1,Rhamnus
61,Rhamnus alaternus,Outer Sunset,1,1,0,Better or Same,1,Rhamnus


In [24]:
df_notna.groupby('genus')['2m_3yr_change'].mean().reset_index().sort('2m_3yr_change').head()

,genus,2m_3yr_change
41,Griselinia,-2.285714
22,Cinnamomum,-2.000000
84,Tibochina,-2.000000
8,Arctostaphylos,-1.666667
4,Albizia,-1.666667


In [25]:
df_notna.groupby('Neighborhood')['2m_3yr_change'].mean().reset_index().sort('2m_3yr_change').head()

,Neighborhood,2m_3yr_change
6,Diamond Heights,-1.000000
14,Hunters Point,-0.833333
17,Laurel Heights,-0.800000
25,OMI,-0.789116
4,Central Sunset,-0.761905


In [26]:
df_notna.head()

,Tree Species,Neighborhood,Survey2mStatus,Survey3yrStatus,2m_3yr_change,2m_3yr_change_category,2m_3yr_change_binary,genus
25,Prunus salicina 'Santa Rosa',Bayview,1,2,-1,Worse,0,Prunus
49,Prunus yedoensis 'Akebono',Mission,1,1,0,Better or Same,1,Prunus
52,Prunus yedoensis 'Akebono',Mission,1,1,0,Better or Same,1,Prunus
58,Rhamnus alaternus,Outer Sunset,1,1,0,Better or Same,1,Rhamnus
61,Rhamnus alaternus,Outer Sunset,1,1,0,Better or Same,1,Rhamnus


In [27]:
df_notna.genus.nunique()

90

In [28]:
df_notna.Neighborhood.nunique()

45

In [29]:
from sklearn import linear_model, decomposition, datasets
from sklearn.pipeline import Pipeline

In [52]:
logistic = linear_model.LogisticRegression()

In [53]:
X_full = pd.get_dummies(df_notna[['genus', 'Neighborhood']], prefix=['genus', 'Neighborhood'])
y_full = df_notna['2m_3yr_change_binary']

n_samples = len(X_full)

X_train = X_full[:int(.6 * n_samples)]
y_train = y_full[:int(.6 * n_samples)]
X_test = X_full[int(.6 * n_samples):]
y_test = y_full[int(.6 * n_samples):]


model = logistic.fit(X_train, y_train)
score = model.score(X_test, y_test)
print(score)

0.542032622334


In [32]:
print(model)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)


In [33]:
X_train.columns

Index([u'genus_Acacia', u'genus_Acer', u'genus_Aesculus', u'genus_Agonis',
       u'genus_Albizia', u'genus_Alnus', u'genus_Aloe', u'genus_Arbutus',
       u'genus_Arctostaphylos', u'genus_Betula', 
       ...
       u'Neighborhood_South Central Sunset',
       u'Neighborhood_Sunnyside/Balboa Terrace',
       u'Neighborhood_Telegraph Hill', u'Neighborhood_Tenderloin',
       u'Neighborhood_Twin Peaks', u'Neighborhood_USF/NoPa',
       u'Neighborhood_Upper Market', u'Neighborhood_Visitacion Valley',
       u'Neighborhood_West Portal',
       u'Neighborhood_Western Addition/Hayes Valley'],
      dtype='object', length=135)

In [39]:
model.coef_

array([[ 0.296557  ,  0.01673612, -0.12187787, -0.2461067 , -0.79716792,
         0.79551709,  0.        ,  0.01049175, -0.54227032,  1.29446096,
         0.        ,  0.75049753, -0.22212636,  0.35643914, -0.49711496,
        -1.13554737,  0.11696896,  0.14114992,  0.06114344, -0.16255893,
         0.34185148,  0.31019987, -0.4707304 , -1.31880227,  0.78604047,
         0.15607039, -0.6387159 ,  0.        ,  0.        , -0.36680769,
         0.25094727,  0.52804554,  0.35741094, -0.05187169, -0.95149646,
        -0.31384521,  0.06675172,  0.94688911, -0.03218854, -0.27227384,
        -1.12352176, -0.77324873,  0.15183015, -0.97995684,  0.64241635,
         0.36668223,  0.07083136,  0.10825646, -0.13247325,  0.82710275,
        -0.80437017, -0.1105604 , -0.54791083,  0.20143163,  0.27148031,
         0.21945127,  0.17960001, -0.1608224 , -0.72398036, -0.36986639,
        -0.14443557,  0.28621584, -0.01616519,  0.53212296, -0.10005336,
        -0.3638231 ,  0.27148031,  0.        ,  0.5

In [35]:
results = [model.coef_,X_train.columns]
results

[array([[ 0.296557  ,  0.01673612, -0.12187787, -0.2461067 , -0.79716792,
          0.79551709,  0.        ,  0.01049175, -0.54227032,  1.29446096,
          0.        ,  0.75049753, -0.22212636,  0.35643914, -0.49711496,
         -1.13554737,  0.11696896,  0.14114992,  0.06114344, -0.16255893,
          0.34185148,  0.31019987, -0.4707304 , -1.31880227,  0.78604047,
          0.15607039, -0.6387159 ,  0.        ,  0.        , -0.36680769,
          0.25094727,  0.52804554,  0.35741094, -0.05187169, -0.95149646,
         -0.31384521,  0.06675172,  0.94688911, -0.03218854, -0.27227384,
         -1.12352176, -0.77324873,  0.15183015, -0.97995684,  0.64241635,
          0.36668223,  0.07083136,  0.10825646, -0.13247325,  0.82710275,
         -0.80437017, -0.1105604 , -0.54791083,  0.20143163,  0.27148031,
          0.21945127,  0.17960001, -0.1608224 , -0.72398036, -0.36986639,
         -0.14443557,  0.28621584, -0.01616519,  0.53212296, -0.10005336,
         -0.3638231 ,  0.27148031,  0.

In [36]:
from sklearn.metrics import roc_curve, auc
from sklearn import preprocessing, metrics, cross_validation
from sklearn.model_selection import cross_val_score


/usr/local/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [45]:
from sklearn.model_selection import cross_val_score

scores = cross_val_score(model, X_full, y_full, cv=5)
scores

array([ 0.56096337,  0.56798796,  0.58082329,  0.56475904,  0.52158635])

In [116]:
from sklearn import metrics

model.fit(X_train, y_train)
pred = model.predict_proba(X_test)[:,1]
fpr, tpr, thresholds = roc_curve(y_test, pred, pos_label = 0)
roc_auc = auc(fpr, tpr)
roc_auc

nan

In [102]:
y_test

35400    0
35489    1
35491    0
35493    1
35497    1
35499    0
35503    0
35505    0
35507    0
35545    1
35547    1
35551    0
35555    0
35559    1
35561    0
35569    1
35571    1
35573    0
35577    0
35581    1
35585    1
35589    0
35991    1
35993    0
35995    0
35997    1
36003    1
36005    1
36007    0
36025    0
        ..
60399    0
60400    1
60405    0
60412    1
60414    1
60415    1
60416    0
60417    1
60418    1
60419    1
60426    1
60439    0
60440    1
60442    0
60443    0
60452    1
60454    1
60456    1
60457    1
60458    1
60459    0
60461    1
60466    1
60467    0
60473    0
60474    1
60494    0
60502    0
60509    1
61300    0
Name: 2m_3yr_change_binary, dtype: object

In [ ]:
X = np.array([[1, 2], [3, 4], [5, 6], [7, 8]])
y = np.array([1, 2, 3, 4])
labels = np.array([0, 0, 2, 2])
label_kfold = LabelKFold(labels, n_folds=2)
len(label_kfold)
2
print(label_kfold)

In [ ]:

X = np.array([[1, 2], [3, 4], [5, 6], [7, 8]])
y = np.array([1, 2, 3, 4])
labels = np.array([0, 0, 2, 2])
label_kfold = LabelKFold(labels, n_folds=2)
len(label_kfold)
2
print(label_kfold)
sklearn.cross_validation.LabelKFold(n_labels=4, n_folds=2)
for train_index, test_index in label_kfold:
print("TRAIN:", train_index, "TEST:", test_index)
X_train, X_test = X[train_index], X[test_index]
y_train, y_test = y[train_index], y[test_index]
print(X_train, X_test, y_train, y_test)

In [ ]:
# Run classifier with cross-validation and plot ROC curves
cv = StratifiedKFold(n_splits=6)
logistic = linear_model.LogisticRegression()

mean_tpr = 0.0
mean_fpr = np.linspace(0, 1, 100)

colors = cycle(['cyan', 'indigo', 'seagreen', 'yellow', 'blue', 'darkorange'])
lw = 2

i = 0
for (train, test), color in zip(cv.split(X_full, y_full), colors):
    probas_ = logistic.fit(X_full[train], y_full[train]).predict_proba(X_full[test])
    # Compute ROC curve and area the curve
    fpr, tpr, thresholds = roc_curve(y_full[test], probas_[:, 1])
    mean_tpr += interp(mean_fpr, fpr, tpr)
    mean_tpr[0] = 0.0
    roc_auc = auc(fpr, tpr)
    plt.plot(fpr, tpr, lw=lw, color=color,
             label='ROC fold %d (area = %0.2f)' % (i, roc_auc))

    i += 1
plt.plot([0, 1], [0, 1], linestyle='--', lw=lw, color='k',
         label='Luck')

mean_tpr /= cv.get_n_splits(X_full, y_full)
mean_tpr[-1] = 1.0
mean_auc = auc(mean_fpr, mean_tpr)
plt.plot(mean_fpr, mean_tpr, color='g', linestyle='--',
         label='Mean ROC (area = %0.2f)' % mean_auc, lw=lw)

plt.xlim([-0.05, 1.05])
plt.ylim([-0.05, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver operating characteristic example')
plt.legend(loc="lower right")
plt.show()